# Natural Language Command & Control for UAV's

## Assignment 2 soln

**Part 1: Theoretical Questions**

Q1. Computational Graphs & Gradients

In PyTorch, gradient accumulation happens by default. When we call loss.backward(), the gradients computed during backpropagation are added to the values already stored in the .grad attribute of the leaf tensors, rather than replacing them. In other words, PyTorch treats the gradient buffers as running sums unless we manually clear them.
Because of this behavior, a typical training loop includes a call to optimizer.zero_grad() before computing the next backward pass. This step resets all stored gradients to zero so that each optimization step is based only on the gradients from the current batch.
If we forget to call zero_grad(), the gradients from previous iterations will continue accumulating across steps. Mathematically, this means the parameter update will use the sum of gradients from multiple batches, which alters the effective learning rate, distorts the optimization dynamics, and can lead to unstable or incorrect training behavior. While intentional accumulation can be useful in techniques like gradient accumulation for small batch training, it must be controlled explicitly.


Q2. Tensors: View vs. Reshape

In PyTorch, both .view() and .reshape() are used to change the shape of a tensor, but they behave differently with respect to memory layout. The .view() method can only be applied when the underlying tensor is stored in a single, contiguous block of memory. If the tensor has been transposed, sliced, or modified in a way that makes it non-contiguous, calling .view() will fail because it cannot simply reinterpret the existing memory as a new shape.

The .reshape() method, however, is more flexible. It first tries to return a view of the tensor (just like .view()), but if the tensor is non-contiguous, it will automatically create a new contiguous copy in memory and then reshape it. This allows the operation to succeed even when the original tensor layout is fragmented.

Because of this behavior, .reshape() is generally considered safer when you are unsure about the tensor’s memory layout or strides. However, when you know for sure that a tensor is already contiguous and you want to avoid any chance of copying data, .view() can be more efficient.

Q3. Device Management (CPU vs. GPU)


In PyTorch, every tensor is associated with a specific device, such as the CPU or GPU. Operations like addition or multiplication can only be performed when all participating tensors are on the same device. If we try to combine a tensor on the CPU with another tensor on the GPU, PyTorch raises a device mismatch error, because it does not implicitly move data across devices. To fix this, we must manually transfer one of the tensors so that both exist on either the CPU or the GPU.

When we move a model to the GPU using model.to('cuda'), all of its parameters and buffers are transferred to GPU memory. However, tensors that are newly created inside the forward method are not automatically placed on the GPU. By default, they are created on the CPU unless we explicitly specify the device. If such tensors interact with the model parameters on the GPU, it will again result in a device mismatch error. Therefore, any intermediate tensors inside the model must also be created on the same device as the model to ensure smooth execution.

### **Part 2: Programming Challenges**

Q4. Tensor Manipulation (The Image Mask)

In [1]:
import torch

def process_images(images):
    return torch.where(images >= 0.5,0.0,1.0)
images = torch.rand(4, 28, 28)
output = process_images(images)
print(output)


tensor([[[1., 0., 1.,  ..., 0., 1., 1.],
         [1., 1., 0.,  ..., 1., 0., 0.],
         [0., 0., 1.,  ..., 1., 1., 0.],
         ...,
         [0., 0., 1.,  ..., 0., 1., 0.],
         [1., 1., 1.,  ..., 1., 0., 1.],
         [0., 0., 1.,  ..., 1., 1., 0.]],

        [[1., 0., 1.,  ..., 1., 0., 1.],
         [0., 0., 1.,  ..., 1., 0., 1.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 0., 1., 1.],
         [1., 1., 0.,  ..., 1., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.]],

        [[0., 1., 0.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 1., 0., 1.],
         [1., 1., 0.,  ..., 0., 0., 1.],
         ...,
         [0., 1., 1.,  ..., 0., 1., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.]],

        [[1., 0., 1.,  ..., 1., 0., 1.],
         [1., 0., 1.,  ..., 0., 1., 1.],
         [1., 0., 0.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 1.,  ..., 1., 1., 0.],
         [1., 1., 1.,  ..., 0., 0., 

Q5. The "Safe" Autograd (Gradient Checking)

In [ ]:
import torch
x = torch.tensor(4.0, requires_grad=True)
y = x**3 + 2*x
y.backward()
print("Gradient:", x.grad)
manual_cal = 3*(4**2) + 2

# sanity check
if x.grad.item() != manual_cal:
    raise ValueError("gradient does not match manual calculation")

print( manual_cal)



**Part 3: Advanced OOP Challenges**

Q6. Class Interaction (Custom Datasets)

In [2]:
import torch
from torch.utils.data import Dataset

class Numdataset(Dataset):
    def __init__(self, numbers):
        self.numbers = numbers

    def __len__(self):
        return len(self.numbers)

    def __getitem__(self, index):
        num = self.numbers[index]
        x = torch.tensor(float(num))
        y = torch.tensor(float(num * 2))
        return x, y

data = Numdataset([1, 2, 3, 4])

print("length ", len(data))
print("item0 ", data[0])
print("item2 ", data[2])




length: 4
item 0: (tensor(1.), tensor(2.))
item 2: (tensor(3.), tensor(6.))


Q7. Model Encapsulation (nn.Module)

In [ ]:
import torch
import torch.nn as nn

class SimpleClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

model = SimpleClassifier()
dummy_input = torch.rand(2, 10)
output = model(dummy_input)

print(output)


Q8. The Training Loop (Logic Integration)


In [ ]:
def train_step(model, inputs, targets, optimizer, criterion):
    # Forward pass
    predictions = model(inputs)

    # Loss calculation
    loss = criterion(predictions, targets)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Return scalar loss value
    return loss.item()

import torch
import torch.nn as nn

# Create model
model = SimpleClassifier()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Loss function
criterion = nn.BCELoss()

# Dummy inputs and targets
inputs = torch.rand(4, 10)
targets = torch.rand(4, 1)

# Run one training step
loss = train_step(model, inputs, targets, optimizer, criterion)

print("Loss:", loss)
